In [ ]:
# Import libraries
from cil.framework import AcquisitionGeometry
from cil.io import NEXUSDataWriter
from cil.processors import Slicer
from cil.utilities.display import show2D
from utils import read_frames, read_extra_frames, download_zenodo

import numpy as np

1) First, we download the dynamic tomographic data from 
[Zenodo](https://zenodo.org/record/3696817#.YGyJMxMzb_Q).

2) There are resolutions available of size 256 x 256 or 512 x 512, (GelPhantomData_b4.mat and GelPhantomData_b2.mat respectively). For the paper, we use the 256x256 resolution for simplicity.

3) Two additional data are provided in GelPhantom_extra_frames.mat with dense sampled measurements from the first time step and the last (18th) time step.

**Note that the pixel size of the detector is wrong. The correct pixel size should be doubled.**

In [ ]:
# Download files from Zenodo
download_zenodo()

In [ ]:
# Read matlab files for the 17 frames
name = "GelPhantomData_b4"
path = "MatlabData/"
file_info = read_frames(path, name)

In [ ]:
# Get sinograms + metadata
sinograms = file_info['sinograms']
frames = sinograms.shape[0]
angles = file_info['angles']
distanceOriginDetector = file_info['distanceOriginDetector']
distanceSourceOrigin = file_info['distanceSourceOrigin']
pixelSize = 2*file_info['pixelSize']
numDetectors = file_info['numDetectors']

In [ ]:
# Create acquisition + image geometries
ag = AcquisitionGeometry.create_Cone2D(source_position = [0, distanceSourceOrigin],
                                       detector_position = [0, -distanceOriginDetector])\
                                    .set_panel(numDetectors, pixelSize)\
                                    .set_channels(frames)\
                                    .set_angles(angles, angle_unit="radian")\
                                    .set_labels(['channel','angle', 'horizontal'])

ig = ag.get_ImageGeometry()
ig.voxel_num_x = 256
ig.voxel_num_y = 256

In [ ]:
# Create the 2D+time acquisition data
data = ag.allocate()

for i in range(frames):
   data.fill(sinograms[i], channel = i) 

In [ ]:
# Create and save Sparse Data with different angular sampling: 18, 36, 72, 360 projections
for i in [1, 5, 10, 20]:
    
    name_proj = "data_{}".format(int(360/i))
    new_data = Slicer(roi={'angle':(0,360,i)})(data)
    ag = new_data.geometry
    ig = ag.get_ImageGeometry()
    ig.voxel_num_x = 256
    ig.voxel_num_y = 256 
    
    show2D(new_data, slice_list = [0,5,10,16], num_cols=4, origin="upper",
                       cmap="inferno", title="Projections {}".format(int(360/i)), size=(25, 20))    
    
    writer = NEXUSDataWriter(file_name = "SparseData/"+name_proj+".nxs",
                         data = new_data)
    writer.write()  
    

In [ ]:
# Read matlab files for the extra frames
name = "GelPhantom_extra_frames"
path = "MatlabData/"
frame1_info = read_extra_frames(path, name, "GelPhantomFrame1_b4")
frame18_info = read_extra_frames(path, name, "GelPhantomFrame18_b4")

# Acquisition geometry for the 1st frame: 720 projections
ag2D_frame1 = AcquisitionGeometry.create_Cone2D(source_position = [0,   frame1_info['distanceSourceOrigin']],
                                       detector_position = [0, -frame1_info['distanceOriginDetector']])\
                                    .set_panel(num_pixels = frame1_info['numDetectors'], pixel_size = 2*frame1_info['pixelSize'])\
                                    .set_angles(frame1_info['angles'])\
                                    .set_labels(['angle', 'horizontal'])

# Acquisition geometry for the 18th frame: 1600 projections
ag2D_frame18 = AcquisitionGeometry.create_Cone2D(source_position = [0,   frame18_info['distanceSourceOrigin']],
                                       detector_position = [0, -frame18_info['distanceOriginDetector']])\
                                    .set_panel(num_pixels = frame18_info['numDetectors'], pixel_size = 2*frame18_info['pixelSize'])\
                                    .set_angles(frame18_info['angles'])\
                                    .set_labels(['angle', 'horizontal'])

# Image geometry is the same
ig = ag2D_frame18.get_ImageGeometry()
ig.voxel_num_x = 256
ig.voxel_num_y = 256

In [ ]:
# Create and save the 2D acquisition data for the extra frames
data = ag2D_frame1.allocate()
data.fill(frame1_info['sinograms'])
show2D(data, cmap="inferno")
 
# Save prescan    
name = "data_prescan_720"
writer = NEXUSDataWriter(file_name = "SparseData/" + name +".nxs",
                     data = data)
writer.write() 

data = ag2D_frame18.allocate()
data.fill(frame18_info['sinograms'])
show2D(data, cmap="inferno")

# Save postscan 
name = "data_postscan_1600"

writer = NEXUSDataWriter(file_name = "SparseData/" + name +".nxs",
                     data = data)
writer.write()